### Import des librairies

In [16]:
import os 
from osgeo import gdal
import matplotlib.pyplot as plt 
import geopandas as gpd

import sys
sys.path.append('/home/onyxia/work')

from libsigma.classification import rasterization

#intégrer dans une variable le chemin d'accès du shp via geopandas 

In [17]:
gdf = gpd.read_file("/home/onyxia/work/data/projet_eval/PI_strates_bretagne_32630.shp")
gdf["strate"].value_counts()


strate
3    121
2     86
4     71
Name: count, dtype: int64

In [26]:
vect = "/home/onyxia/work/data/projet_eval/PI_strates_bretagne_32630.shp"
rst = "/home/onyxia/work/data/projet_eval/bretagne_23-24_B02.tif"
result = "/home/onyxia/work/result/test_img.tif"
strat = "strate"